# 제주도 식당의 리뷰와 정보들을 네이버에서 크롤링해오기
- 제주도식당_전처리.csv를 사용

In [1]:
import pandas as pd
import random
import time
import re
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver  # 동적크롤링
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

### 크롤링에 필요한 데이터 불러오기

In [2]:
df = pd.read_csv("Data/제주도식당_전처리.csv", index_col=False)

In [3]:
df.head()

,사업장명,업종구분대분류,업종구분소분류,소재지전체주소,도로명전체주소,lat,lng
0,탑동에이치지,일반음식점,기타,제주특별자치도 제주시 삼도이동 1120-2,"제주특별자치도 제주시 무근성안길 16, 1층 (삼도이동)",33.5151128,126.5202001
1,에릭스에스프레소,일반음식점,기타,제주특별자치도 제주시 구좌읍 세화리 1397-6번지,"제주특별자치도 제주시 구좌읍 구좌로 77, 1층",33.5214225,126.8588743
2,일품순두부한림점,일반음식점,한식,제주특별자치도 제주시 한림읍 대림리 1845-2번지,"제주특별자치도 제주시 한림읍 한림상로 237, 1층",33.4192601,126.2674304
3,김복남맥주제주아라점,일반음식점,식육(숯불구이),제주특별자치도 제주시 아라일동 6139,"제주특별자치도 제주시 중앙로 581, 에이동 1층 (아라일동)",33.4714278,126.5457848
4,봉플라봉뱅,일반음식점,기타,제주특별자치도 제주시 연동 312-57번지 정인하우스,"제주특별자치도 제주시 문송1길 6-1, 정인하우스 1층 (연동)",33.4879976,126.4978933


### 크롤링에 필요한 컬럼만 뽑아서 저장하기

In [4]:
scrapy_df = df[["사업장명", "도로명전체주소"]]

In [5]:
scrapy_df.head()

,사업장명,도로명전체주소
0,탑동에이치지,"제주특별자치도 제주시 무근성안길 16, 1층 (삼도이동)"
1,에릭스에스프레소,"제주특별자치도 제주시 구좌읍 구좌로 77, 1층"
2,일품순두부한림점,"제주특별자치도 제주시 한림읍 한림상로 237, 1층"
3,김복남맥주제주아라점,"제주특별자치도 제주시 중앙로 581, 에이동 1층 (아라일동)"
4,봉플라봉뱅,"제주특별자치도 제주시 문송1길 6-1, 정인하우스 1층 (연동)"


### 네이버 지도창에서 검색할 때 정확도를 높이기 위해 도로명 주소의 , 뒤에는 제거
> 이유는 네이버 지도탭에서 검색시 "사업장명" + "도로명전체주소" 형식으로 검색하면 정확히 원하는 식당이 뜨는 빈도수가 높기 때문에   
도로명전체주소가 필요한 것인데, 문제는 도로명주소에 상세주소가 적혀있으면 검색이 안될 확률이 높았음   
때문에 ','뒤에 있는 상세주소를 제거할 필요가 있음


In [6]:
scrapy_df['도로명전체주소'] = scrapy_df['도로명전체주소'].str.split(',', n=1).str[0]

In [7]:
scrapy_df.head(40)

,사업장명,도로명전체주소
0,탑동에이치지,제주특별자치도 제주시 무근성안길 16
1,에릭스에스프레소,제주특별자치도 제주시 구좌읍 구좌로 77
2,일품순두부한림점,제주특별자치도 제주시 한림읍 한림상로 237
3,김복남맥주제주아라점,제주특별자치도 제주시 중앙로 581
4,봉플라봉뱅,제주특별자치도 제주시 문송1길 6-1
5,멜꽃삼,제주특별자치도 제주시 중앙로 573
6,가빈거래처,제주특별자치도 제주시 산천단동2길 13
7,스톤아일랜드탭하우스,제주특별자치도 제주시 삼무로7길 16
8,옹포분식,제주특별자치도 제주시 한림읍 월계로 15
9,연두랑고갈비,제주특별자치도 제주시 독짓골2길 10


---
## 크롤링하기

In [8]:
# 크롬 드라이버 실행
# chrome_options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [9]:
# # 리뷰를 추출해주는 함수
# def extract_review(reviewCount: int):
#     # 리뷰 추출
#     rev = []  # 추출한 리뷰 저장
#     rangeNum = 5
#     if reviewCount < 11:
#         rangeNum = 11
#         time.sleep(1)
#     elif 10 <= reviewCount < 21:
#         rangeNum = 21
#         driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
#         time.sleep(1)
#     elif reviewCount >= 20:
#         rangeNum = 31
#         driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
#         time.sleep(1)
#         driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
#         time.sleep(1)

#     for i in range(1, rangeNum): # 더보기 눌러놓고 30개 가져오기
#         try:  # 사진 있는 후기는 div 3번째에 텍스트 위치
#             driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
#             # print('리뷰가 저장됨')
#             # time.sleep(0.5)
#             comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰
#             # print('리뷰가 저장됨')
#             rev.append(comment)
#         except: # 사진 없는 후기는 div 2번째에 텍스트가 위치
#             try: # 리뷰에 글이 없는경우에 XPATH가 달라짐으로 try - except 추가
#                 driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
#                 comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
#                 # print('리뷰가 저장됨')
#                 rev.append(comment)
#             except:
#                 # print('이상한 리뷰임')
#                 rev.append(pd.NA)

#     # print('더 이상 리뷰가 없음')
#     return rev

#### 버튼에있는 방문 리뷰수가 아닌 실제 방문 리뷰수를 가져오는게 불가능해서    
#### for문을 돌려가며 더보기 버튼을 누르는 방식으로 변경

In [10]:
# # 리뷰를 추출해주는 함수
# def extract_review(listsCount: int):
#     rev = []  # 추출한 리뷰 저장
#     for i in range(1, 11): # 더보기 누르지 않은 상태로 최대 10개
#         # try:  
#             # if listsCount == 1: # 블로그 리뷰가 없을 시에 달라지는 XPATH값 예외처리 추가
#             #     driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[{i}]/div[2]/a').send_keys(Keys.ENTER) 
#             #     comment = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[{i}]/div[2]/a/span').text  # 리뷰
#             #     time.sleep(random.uniform(0.2, 0.5))
#             #     rev.append(comment) 
#             # else:
#             #     driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[3]/a').send_keys(Keys.ENTER) 
#             #     comment = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[3]/a/span').text  # 리뷰
#             #     time.sleep(random.uniform(0.2, 0.5))
#             #     rev.append(comment)

#         #    for i in range(1, 11): # 더보기 누르지 않은 상태로 최대 10개
#             try:  # 사진 있는 후기는 div 3번째에 텍스트 위치
#                 driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
#                 # print('리뷰가 저장됨')
#                 time.sleep(random.uniform(1,2))
#                 comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰
#                 # print('리뷰가 저장됨')
#                 rev.append(comment)
#             except: # 사진 없는 후기는 div 2번째에 텍스트가 위치
#                 try: # 리뷰에 글이 없는경우에 XPATH가 달라짐으로 try - except 추가
#                     driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
#                     time.sleep(random.uniform(1,2))
#                     comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
#                     # print('리뷰가 저장됨')
#                     rev.append(comment)
#                 except:
#                     # print('이상한 리뷰임')
#                     rev.append("")

#         # except: 
#         #     try: 
#         #         if listsCount == 1: # 블로그 리뷰가 없을 시에 달라지는 XPATH값 예외처리 추가
#         #             driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[{i}]/div[2]/a').send_keys(Keys.ENTER) 
#         #             comment = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[{i}]/div[2]/a/span').text  # 리뷰
#         #             time.sleep(random.uniform(0.2, 0.5))
#         #             rev.append(comment)                    
#         #         else:
#         #             driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[3]/a').send_keys(Keys.ENTER) 
#         #             comment = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[3]/a/span').text  # 리뷰
#         #             time.sleep(random.uniform(0.2, 0.5))
#         #             rev.append(comment)
#         #     except:
#         #         rev.append("")

#     # 더 많은 리뷰가 있는지 확인하고, 있을 경우에는 추가 리뷰를 가져오기 위해 "더보기" 버튼 클릭
#     try:
#         if listsCount == 1:
#             driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
#         else:
#             driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)   
#         time.sleep(1)
#         for i in range(11, 21):
#             try:  
#                 if listsCount == 1: # 블로그 리뷰가 없을 시에 달라지는 XPATH값 예외처리 추가
#                     driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[{i}]/div[2]/a').send_keys(Keys.ENTER)  
#                     comment = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[{i}]/div[2]/a/span').text  # 리뷰
#                     time.sleep(random.uniform(0.2, 0.5))
#                     rev.append(comment) 
#                 else:
#                     driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[3]/a').send_keys(Keys.ENTER) 
#                     comment = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[3]/a/span').text  # 리뷰
#                     time.sleep(random.uniform(0.2, 0.5))
#                     rev.append(comment)

#             except: 
#                 try: 
#                     if listsCount == 1: # 블로그 리뷰가 없을 시에 달라지는 XPATH값 예외처리 추가
#                         driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[{i}]/div[2]/a').send_keys(Keys.ENTER)  
#                         comment = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[{i}]/div[2]/a/span').text  # 리뷰
#                         time.sleep(random.uniform(0.2, 0.5))
#                         rev.append(comment) 
#                     else:
#                         driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[3]/a').send_keys(Keys.ENTER) 
#                         comment = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[3]/a/span').text  # 리뷰
#                         time.sleep(random.uniform(0.2, 0.5))
#                         rev.append(comment)

#                 except:
#                     rev.append("")

#     except:
#         print('더 이상 리뷰가 없음')

#     # 추가로 한 번 더 더보기 버튼을 클릭하여 최대 30개의 리뷰 가져오기
#     try:
#         time.sleep(random.uniform(0.3, 0.5)) 
#         if listsCount == 1:
#             driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
#         else:
#             driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)   
#         time.sleep(1)
#         for i in range(21, 31):
#             try:  
#                 if listsCount == 1: # 블로그 리뷰가 없을 시에 달라지는 XPATH값 예외처리 추가
#                     driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[{i}]/div[2]/a').send_keys(Keys.ENTER)  
#                     comment = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[{i}]/div[2]/a/span').text  # 리뷰
#                     time.sleep(random.uniform(0.2, 0.5))
#                     rev.append(comment) 
#                 else:
#                     driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[3]/a').send_keys(Keys.ENTER) 
#                     comment = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[3]/a/span').text  # 리뷰
#                     time.sleep(random.uniform(0.2, 0.5))
#                     rev.append(comment)

#             except: 
#                 try: 
#                     if listsCount == 1: # 블로그 리뷰가 없을 시에 달라지는 XPATH값 예외처리 추가
#                         driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[{i}]/div[2]/a').send_keys(Keys.ENTER)  
#                         comment = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[{i}]/div[2]/a/span').text  # 리뷰
#                         time.sleep(random.uniform(0.2, 0.5))
#                         rev.append(comment) 
#                     else:
#                         driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[3]/a').send_keys(Keys.ENTER) 
#                         comment = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[3]/a/span').text  # 리뷰
#                         time.sleep(random.uniform(0.2, 0.5))
#                         rev.append(comment)

#                 except:
#                     rev.append("")

#     except:
#         print('더 이상 리뷰가 없음')

#     return rev

In [11]:
# 리뷰를 추출해주는 함수
def extract_review(listsCount: int):
    time.sleep(random.uniform(3, 3.5))
    rev = []  # 추출한 리뷰 저장
    for i in range(1, 11): 
        try:  # 사진 있는 후기는 div 3번째에 텍스트 위치
            if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
                driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
                time.sleep(0.1)
                comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰

                rev.append(comment)
            else:
                driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
                time.sleep(0.1)
                comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰

                rev.append(comment)
        except: # 사진 없는 후기는 div 2번째에 텍스트가 위치
            try: # 리뷰에 글이 없는경우에 XPATH가 달라짐으로 try - except 추가
                if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
                    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
                    time.sleep(0.1)
                    comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                    
                    rev.append(comment)
                else:
                    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
                    time.sleep(0.1)
                    comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                    
                    rev.append(comment)
            except:
                rev.append("")

    # 더 많은 리뷰가 있는지 확인하고, 있을 경우에는 추가 리뷰를 가져오기 위해 "더보기" 버튼 클릭
    try:
        if listsCount == 1:
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
        else:
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)   
        time.sleep(0.5)
        for i in range(11, 21):
            try:  # 사진 있는 후기는 div 3번째에 텍스트 위치
                if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
                    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
                    time.sleep(0.1)
                    comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰

                    rev.append(comment)
                else:
                    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
                    time.sleep(0.1)
                    comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰

                    rev.append(comment)
            except: # 사진 없는 후기는 div 2번째에 텍스트가 위치
                try: # 리뷰에 글이 없는경우에 XPATH가 달라짐으로 try - except 추가
                    if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
                        driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
                        time.sleep(0.1)
                        comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                        
                        rev.append(comment)
                    else:
                        driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
                        time.sleep(0.1)
                        comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                        
                        rev.append(comment)
                except:
                    rev.append("")

    except:
        print('더 이상 리뷰가 없음')
    
    try:
        if listsCount == 1:
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
        else:
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)   
        time.sleep(0.5)
        for i in range(21, 31):
            try:  # 사진 있는 후기는 div 3번째에 텍스트 위치
                if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
                    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
                    time.sleep(0.1)
                    comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰

                    rev.append(comment)
                else:
                    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
                    time.sleep(0.1)
                    comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰

                    rev.append(comment)
            except: # 사진 없는 후기는 div 2번째에 텍스트가 위치
                try: # 리뷰에 글이 없는경우에 XPATH가 달라짐으로 try - except 추가
                    if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
                        driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
                        time.sleep(0.1)
                        comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                        
                        rev.append(comment)
                    else:
                        driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
                        time.sleep(0.1)
                        comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                        
                        rev.append(comment)

                except:
                    rev.append("")

    except:
        print('더 이상 리뷰가 없음')
    
    
    return rev




In [12]:
count = 0 # 성공한 크롤링 카운트
failedCount = 0 # 실패한 크롤링 카운트

crawlingReviewData = [] # 식당 리뷰를 저장할 리스트
crawlingImageData = [] # 식당 이미지를 저장할 리스트
crawlingCategoryData = [] # 식당 업종분류를 저장할 리스트
crawlingVisitCountData = [] # 식당 방문자리뷰 수를 저장할 리스트
crawlingBlogCountData = [] # 식당 블로그리뷰 수를 저장할 리스트
crawlingRatingData = [] # 식당 평점을 저장할 리스트
errorResult = ""

result_df = pd.DataFrame()
# range(len(scrapy_df))
# range(1, 2)
for i in range(2001, 3001):
    url = f"https://map.naver.com/v5/search/{scrapy_df['사업장명'][i]} {scrapy_df['도로명전체주소'][i]}"
    # url = f"https://map.naver.com/v5/search/걸작떡볶이제주노형점 제주특별자치도 제주시 노형9길 7"
    # url = f"https://map.naver.com/v5/search/신촌도시락 제주 제주시 조천읍 신북로 88"
    # url = f"https://map.naver.com/v5/search/가시림 카페 제주 서귀포시 표선면 녹산로 5번길 171"
    
    driver.get(url)
    time.sleep(random.uniform(2.0, 2.5))
    try: 
        driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동
        time.sleep(random.uniform(1.5, 2.0))
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 이미지 3장 뽑아서 저장하기
        images = []  # 이미지 링크를 저장할 리스트
        for i in range(1, 4):
            image_element = soup.select_one(f"#ibu_{i}")  # ibu_1 ~ 3 이런식으로 이미지의 id가 갯수별로 형성됨
            if not image_element:
                continue  # 이미지 요소가 없으면 for 루프 종료
            style = image_element["style"]
            background_image = style.split("url(")[1].split(")")[0]
            images.append(background_image)  # 이미지 링크를 리스트에 추가

        # 음식 카테고리 뽑기 (버튼이라서 CSS_SELECTOR를 써야함)
        foodCategory = driver.find_element(By.CSS_SELECTOR, '#_title > div > span.DJJvD').text


        # 리뷰 탭을 선택하기 전에 버튼 갯수 파악하기 (별점 4.7, 방문자리뷰 100, 블로그리뷰 120) 이런식으로 된곳
        lists = soup.select('.PXMot')  # 버튼들의 class = PXMot
        errorResult = "검색 결과 없음"
        if len(lists) == 3: # (별점이 있는 경우) 별점/방문자리뷰/블로그리뷰 순일때 방문자리뷰는 두번째에 위치=span[2]
            # 별점 수, 리뷰 수 긁어오기
            ratingText = soup.select_one('.PXMot.LXIwF').text # 별점의 class = .PXMot.LXIwF
            visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text
            blogText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(3) > a').text

            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a').send_keys(Keys.ENTER) # 방문자 리뷰
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
            time.sleep(1)
        elif len(lists) == 2: # (별점이 없는 경우) 방문자리뷰/블로그리뷰 순일때 방문자리뷰는 첫번째에 위치=span[1]
            # 리뷰 수 긁어오기                                 
            visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(1) > a').text
            blogText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text
            
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/a').send_keys(Keys.ENTER) # 방문자 리뷰 클릭
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
            time.sleep(1)
        elif len(lists) == 1: # 블로그 리뷰가 없는 경우 
            visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span > a').text
            blogText = "블로그리뷰 0"

            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span/a').send_keys(Keys.ENTER) # 방문자 리뷰 클릭
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
            time.sleep(1) 
        else:
            errorResult = "크롤링 불가"
                                        
        
        # time.sleep(random.uniform(0.8, 1.0))
        # review = extract_review(int(re.sub(r'\D', '', visitText)))  # 정규표현식을 사용하여 숫자만 추출
        review = extract_review(len(lists))

        crawlingReviewData.append(review)
        crawlingImageData.append(images)
        crawlingCategoryData.append(foodCategory)
        crawlingRatingData.append(ratingText.replace("별점", ""))
        crawlingVisitCountData.append(int(re.sub(r'\D', '', visitText)))  # 정규표현식을 사용하여 숫자만 추출
        crawlingBlogCountData.append(int(re.sub(r'\D', '', blogText)))
        count += 1
        print("----------------------------------")
        print(f'크롤링 성공! 성공: {count}, 실패: {failedCount}')
        ## 별점이 있고 없고에 따라 공백 데이터 추가해주는거 필요
    except:
        crawlingReviewData.append('x')
        crawlingImageData.append('x')
        crawlingCategoryData.append('x')
        crawlingRatingData.append('x')
        crawlingVisitCountData.append("x")
        crawlingBlogCountData.append("x")
        failedCount += 1
        print("----------------------------------")
        print(f'크롤링 실패! 성공: {count}, 실패: {failedCount}, 이유: {errorResult}')
        

    result_df = pd.DataFrame({'foodCategory': crawlingCategoryData, 'images': crawlingImageData, 'rating': crawlingRatingData, 'visitReviewCount': crawlingVisitCountData, 'blogReviewCount': crawlingBlogCountData,'review': crawlingReviewData})


----------------------------------
크롤링 실패! 성공: 0, 실패: 1, 이유: 
----------------------------------
크롤링 성공! 성공: 1, 실패: 1
----------------------------------
크롤링 성공! 성공: 2, 실패: 1
----------------------------------
크롤링 실패! 성공: 2, 실패: 2, 이유: 검색 결과 없음
----------------------------------
크롤링 성공! 성공: 3, 실패: 2
----------------------------------
크롤링 성공! 성공: 4, 실패: 2
----------------------------------
크롤링 성공! 성공: 5, 실패: 2
----------------------------------
크롤링 성공! 성공: 6, 실패: 2
----------------------------------
크롤링 실패! 성공: 6, 실패: 3, 이유: 검색 결과 없음
----------------------------------
크롤링 성공! 성공: 7, 실패: 3
----------------------------------
크롤링 실패! 성공: 7, 실패: 4, 이유: 검색 결과 없음
----------------------------------
크롤링 실패! 성공: 7, 실패: 5, 이유: 검색 결과 없음
----------------------------------
크롤링 실패! 성공: 7, 실패: 6, 이유: 검색 결과 없음
----------------------------------
크롤링 실패! 성공: 7, 실패: 7, 이유: 검색 결과 없음
----------------------------------
크롤링 성공! 성공: 8, 실패: 7
----------------------------------
크롤링 성공! 성공: 9, 실패: 7
--------------

TimeoutException: Message: timeout: Timed out receiving message from renderer: -0.000
  (Session info: chrome=119.0.6045.105)
Stacktrace:
	GetHandleVerifier [0x00007FF71DC082B2+55298]
	(No symbol) [0x00007FF71DB75E02]
	(No symbol) [0x00007FF71DA305AB]
	(No symbol) [0x00007FF71DA1E997]
	(No symbol) [0x00007FF71DA1E7A8]
	(No symbol) [0x00007FF71DA1CF0E]
	(No symbol) [0x00007FF71DA1D9CE]
	(No symbol) [0x00007FF71DA2A20F]
	(No symbol) [0x00007FF71DA3B551]
	(No symbol) [0x00007FF71DA3FC1A]
	(No symbol) [0x00007FF71DA1E032]
	(No symbol) [0x00007FF71DA3B42A]
	(No symbol) [0x00007FF71DAAA9C8]
	(No symbol) [0x00007FF71DA91E83]
	(No symbol) [0x00007FF71DA6670A]
	(No symbol) [0x00007FF71DA67964]
	GetHandleVerifier [0x00007FF71DF80AAB+3694587]
	GetHandleVerifier [0x00007FF71DFD728E+4048862]
	GetHandleVerifier [0x00007FF71DFCF173+4015811]
	GetHandleVerifier [0x00007FF71DCA47D6+695590]
	(No symbol) [0x00007FF71DB80CE8]
	(No symbol) [0x00007FF71DB7CF34]
	(No symbol) [0x00007FF71DB7D062]
	(No symbol) 

In [ ]:
# 출력 옵션 설정 (컬럼 너비 조정)
# pd.set_option('display.max_colwidth', None) # 길이 제한 해제
# pd.set_option('display.max_colwidth', 50) # 길이 기본 값
# 백수당 제주특별자치도 제주시 구좌읍 월정1길 63
# 낙지육회제주탕탕이 제주특별자치도 제주시 진남로6길 38-1 (화북일동)
# 사라숯불구이 제주특별자치도 제주시 서사로 109
# 24시국수회관 제주특별자치도 제주시 남성로 122-1 (삼도일동)

In [ ]:
result_df['review']

In [ ]:
result_df.tail()

In [ ]:
result_df.to_csv("Data/제주도식당_크롤링_3000.csv", index=False)